In [142]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [143]:
data = {
    "date": [
        "1964-05-14", "1965-05-03", "1966-05-17", "1967-05-10", "1968-05-19",
        "1969-05-20", "1970-05-15", "1971-05-14", "1972-05-11", "1973-05-09",
        "1974-05-13", "1975-05-05", "1976-05-10", "1977-05-12", "1978-05-15",
        "1979-05-11", "1980-05-08", "1981-05-17", "1982-05-10", "1983-04-30",
        "1984-05-07", "1985-05-18", "1986-05-10", "1987-05-16", "1988-05-09",
        "1989-04-27", "1990-04-27", "1991-05-02", "1992-05-08", "1993-05-07",
        "1994-04-30", "1995-05-07", "1996-05-20", "1997-05-16", "1998-05-08",
        "1999-04-29", "2000-04-29", "2001-05-05", "2002-04-29", "2003-05-09",
        "2004-05-01", "2005-05-06", "2006-05-01", "2007-04-27", "2008-05-03",
        "2009-05-03", "2010-05-04", "2011-04-26", "2012-05-07", "2013-05-01",
        "2014-04-24", "2015-04-27", "2016-04-29", "2017-05-17", "2018-05-06",
        "2019-04-29", "2020-05-06", "2021-04-26", "2022-05-03", "2023-05-01",
        "2024-05-07"
    ]
}

In [144]:
df = pd.read_csv("Rovaniemi Apukka_ 1.1.1844 - 15.4.2025_1a7f08c2-4d3f-4e1e-88fb-5851b6f825af.csv", encoding='utf-8')

agg_func = {'Sademäärä [mm]': 'sum', 'Lumensyvyys [cm]': 'sum', 'Ilman keskilämpötila [°C]': 'sum', 'Maanpintaminimi [°C]': 'sum', 'Ylin lämpötila [°C]': 'sum', 'Alin lämpötila [°C]': 'sum'}

df = df.loc[df['Vuosi'] >= 1964]

df = df.drop(columns=['Havaintoasema'])
df = df.apply(pd.to_numeric, errors='coerce')
df = df.groupby(['Vuosi', 'Kuukausi', 'Päivä']).aggregate(agg_func).reset_index()
df

,Vuosi,Kuukausi,Päivä,Sademäärä [mm],Lumensyvyys [cm],Ilman keskilämpötila [°C],Maanpintaminimi [°C],Ylin lämpötila [°C],Alin lämpötila [°C]
0,1964,1,1,-1.0,45.0,-11.8,-9.5,-2.7,-18.2
1,1964,1,2,0.4,45.0,-12.7,-20.6,-9.9,-21.7
2,1964,1,3,0.8,45.0,-3.0,-10.4,0.3,-10.0
3,1964,1,4,-1.0,45.0,-0.8,-7.3,1.2,-1.1
4,1964,1,5,-1.0,45.0,0.2,-8.1,2.9,-5.8
...,...,...,...,...,...,...,...,...,...
22381,2025,4,11,2.1,21.0,-4.5,-9.1,-0.9,-8.5
22382,2025,4,12,0.7,21.0,-3.2,-5.8,0.8,-5.5
22383,2025,4,13,-1.0,24.0,0.8,-11.9,6.3,-5.3
22384,2025,4,14,2.2,22.0,3.8,-4.9,7.7,-2.7


In [ ]:
detached = []
days_of_year = []

years = df.iloc[:, [0]].values
months = df.iloc[:, [1]].values
days = df.iloc[:, [2]].values

# "1964-05-14"

for i in range(len(df)):
    if months[i] < 10:
        month = f"0{str(months[i][0])}"
    else:
        month = str(months[i][0])

    if days[i] < 10:
        day = f"0{str(days[i][0])}"
    else:
        day = str(days[i][0])

    date = f"{str(years[i][0])}-{month}-{day}"

    if date in data['date']:
        detached.append(1)
    else:
        detached.append(0)

    day_of_year = datetime(years[i][0], months[i][0], days[i][0]).timetuple().tm_yday

    print(day_of_year)

    

df['detached'] = detached

df.loc[df['detached'] == 1]


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
features = ['date', 'Sademäärä [mm]', 'Lumensyvyys [cm]', 'Ilman keskilämpötila [°C]', 'Maanpintaminimi [°C]', 'Ylin lämpötila [°C]', 'Alin lämpötila [°C]']
label = ['detached', 'date']

X, y = df[features].values, df[label].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f"Mean squared error: {mse:.2f}")
print(f"R2 score: {r2:.2f}")

ValueError: could not convert string to float: '2005-07-19'

In [ ]:
clf = DecisionTreeClassifier(max_depth=5)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

print()
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6696
           1       0.00      0.00      0.00        20

    accuracy                           1.00      6716
   macro avg       0.50      0.50      0.50      6716
weighted avg       0.99      1.00      1.00      6716

[[6693    3]
 [  20    0]]

0.9965753424657534


In [ ]:
print(y_pred)

[0 0 0 ... 0 0 0]
